In [ ]:
import pymongo
from bson import ObjectId

Connection to MongoDB

In [ ]:
class Connection:
    """Class that connects to the database creating all necessary methods for connection and closing connection, as well as the base database for adding new HTS records and the string_dict collection too
    """

    def __init__(self, db_path: str):
        """_init_ function of the class, defines the connection variables

        Args:
            db_path (str): Path to the database connection on MongoDB
        """

        self.client = pymongo.MongoClient(db_path)
        self.db = self.client['hts']
        self.collection_records = self.db['hts_records']
        self.collection_string_dict = self.db['string_dict']

Definition of pipelines and criterias for pymongo queries

In [ ]:
def pipelineHTSSearch(hts_chap_parsed: str):

    criteria = {'header': hts_chap_parsed}

    pipeline = [
        {
            '$match': criteria
        },
        {
            '$unwind': '$data'
        },
        
    ]

FUnction definition for executing the queries with the Connection class

In [ ]:
def queryHTS(pipeline: list, connection: pymongo.collection.Collection):

